In [1]:
import time

import requests

import bs4
from bs4 import BeautifulSoup

In [2]:
us_embassy_main_url = 'https://www.usembassy.gov/post-sitemap.xml'
us_embassy_main_request = requests.get(us_embassy_main_url)
us_embassy_main_request

<Response [200]>

In [3]:
us_embassy_main_html = us_embassy_main_request.content
us_embassy_main_soup = BeautifulSoup(us_embassy_main_html, 'lxml')

# Some Comment

In [4]:
embassy_url_list = [loc.string for loc in us_embassy_main_soup.find_all('loc')]
embassy_url_list[:5]

['https://www.usembassy.gov/south-africa/',
 'https://www.usembassy.gov/indonesia/',
 'https://www.usembassy.gov/malaysia/',
 'https://www.usembassy.gov/laos/',
 'https://www.usembassy.gov/marshall-islands/']

In [9]:
list(enumerate(embassy_url_list))

[(0, 'https://www.usembassy.gov/south-africa/'),
 (1, 'https://www.usembassy.gov/indonesia/'),
 (2, 'https://www.usembassy.gov/malaysia/'),
 (3, 'https://www.usembassy.gov/laos/'),
 (4, 'https://www.usembassy.gov/marshall-islands/'),
 (5, 'https://www.usembassy.gov/samoa/'),
 (6, 'https://www.usembassy.gov/thailand/'),
 (7, 'https://www.usembassy.gov/vietnam/'),
 (8, 'https://www.usembassy.gov/austria/'),
 (9, 'https://www.usembassy.gov/belarus/'),
 (10, 'https://www.usembassy.gov/czech-republic/'),
 (11, 'https://www.usembassy.gov/denmark/'),
 (12, 'https://www.usembassy.gov/france/'),
 (13, 'https://www.usembassy.gov/ireland/'),
 (14, 'https://www.usembassy.gov/italy/'),
 (15, 'https://www.usembassy.gov/latvia/'),
 (16, 'https://www.usembassy.gov/malta/'),
 (17, 'https://www.usembassy.gov/poland/'),
 (18, 'https://www.usembassy.gov/romania/'),
 (19, 'https://www.usembassy.gov/sweden/'),
 (20, 'https://www.usembassy.gov/united-kingdom/'),
 (21, 'https://www.usembassy.gov/jordan/'),
 (

In [17]:
embassy_request =requests.get(embassy_url_list[176])
embassy_html = embassy_request.content
embassy_soup = BeautifulSoup(embassy_html, 'lxml')
embassy_soup.find_all("meta")

[<meta charset="utf-8"/>,
 <meta content="ie=edge" http-equiv="x-ua-compatible"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/>,
 <meta content="en_US" property="og:locale"/>,
 <meta content="article" property="og:type"/>,
 <meta content="Holy See | USEmbassy.gov" property="og:title"/>,
 <meta content="U.S. Embassy to the Holy See Via Sallustiana, 49 00187, Rome, Italy Phone: (+39)-06-4674-1" property="og:description"/>,
 <meta content="http://www.usembassy.gov/holy-see/" property="og:url"/>,
 <meta content="USEmbassy.gov" property="og:site_name"/>,
 <meta content="2016-10-06T17:58:52+00:00" property="article:published_time"/>,
 <meta content="2021-03-17T20:12:49+00:00" property="article:modified_time"/>,
 <meta content="summary_large_image" name="twitter:card"/>,
 <meta content="Written by" name="twitter:label1"/>,
 <meta content="bottsc" name="twitt

In [34]:
import re
list(map(lambda x: re.match("https://[*]+\.usembassy*", x['href']), embassy_soup.find_all('a')))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [95]:
def embassy_url_prefix(x):
    a = re.match("(http|https)://(?!www).*\.(usmission|usembassy|usconsulate).*", x['href'])
    return a.group() if a else None

In [52]:
embassy_url_map = {}
for embassy in embassy_url_list:
    country_name = embassy.split('/')[-2]
    print(embassy.split('/')[-2])
    embassy_request = requests.get(embassy)
    embassy_html = embassy_request.content
    embassy_soup = BeautifulSoup(embassy_html, 'lxml')
    # cities = embassy_soup.find(class_='cityname1')
    # country_website = '/'.join(cities.a['href'].split('/')[:3])
    country_website_list = list(
        filter(
            None,
            list(
                map(
                    embassy_url_prefix,
                    embassy_soup.find_all('a')
                )
            )
        )
    )
    if len(country_website_list)>0:
        country_website = country_website_list[0]
        country_website = '/'.join(country_website.split('/')[:3])
        print(country_website)
        embassy_url_map[country_name] = country_website
    time.sleep(2)

south-africa
https://za.usembassy.gov
indonesia
https://id.usembassy.gov
malaysia
https://my.usembassy.gov
laos
https://la.usembassy.gov
marshall-islands
https://mh.usembassy.gov
samoa
https://ws.usembassy.gov
thailand
https://th.usembassy.gov
vietnam
https://vn.usembassy.gov
austria
https://at.usembassy.gov
belarus
https://by.usembassy.gov
czech-republic
https://cz.usembassy.gov
denmark
https://dk.usembassy.gov
france
https://fr.usembassy.gov
ireland
https://ie.usembassy.gov
italy
https://it.usembassy.gov
latvia
https://lv.usembassy.gov
malta
https://mt.usembassy.gov
poland
https://pl.usembassy.gov
romania
https://ro.usembassy.gov
sweden
https://se.usembassy.gov
united-kingdom
https://uk.usembassy.gov
jordan
https://jo.usembassy.gov
saudi-arabia
https://sa.usembassy.gov
bangladesh
https://bd.usembassy.gov
kazakhstan
https://kz.usembassy.gov
pakistan
https://pk.usembassy.gov
sri-lanka
https://lk.usembassy.gov
argentina
https://ar.usembassy.gov
chile
https://cl.usembassy.gov
ecuador
htt

In [86]:
def d(embassy_url):
    country_name = embassy_url.split('/')[-2]
    return country_name
missing_embassies = []
# set(map(d, embassy_url_list)) - set(embassy_url_map.keys())
for i, country in list(enumerate(embassy_url_list)):
    if d(country) not in set(embassy_url_map.keys()):
        missing_embassies.append(country)
missing_embassies

['https://www.usembassy.gov/hong-kong/',
 'https://www.usembassy.gov/guatemala/',
 'https://www.usembassy.gov/bermuda/',
 'https://www.usembassy.gov/curacao/',
 'https://www.usembassy.gov/usau/',
 'https://www.usembassy.gov/taiwan-2/']

In [96]:
url_request = requests.get('https://www.usembassy.gov/guatemala')
url_html = url_request.content
url_soup = BeautifulSoup(url_html, 'lxml')
list(map(embassy_url_prefix, url_soup.find_all('a')))

[None,
 None,
 None,
 None,
 None,
 'http://gt.usembassy.gov/embassy/',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [97]:
for url in missing_embassies:
    country_name = d(url)
    url_request = requests.get(url)
    url_html = url_request.content
    url_soup = BeautifulSoup(url_html, 'lxml')
    url_links = list(
        filter(
            None,
            list(
                map(
                    embassy_url_prefix,
                    url_soup.find_all('a')
                )
            )
        )
    )
    print(country_name)
    if len(url_links)>0:
        country_website = url_links[0]
        country_website = '/'.join(country_website.split('/')[:3])
        print(country_website)
        embassy_url_map[country_name] = country_website
    time.sleep(2)

hong-kong
https://hk.usconsulate.gov
guatemala
http://gt.usembassy.gov
bermuda
https://bm.usconsulate.gov
curacao
https://cw.usconsulate.gov
usau
taiwan-2


In [98]:
set(map(d, embassy_url_list)) - set(embassy_url_map.keys())

{'taiwan-2', 'usau'}

In [99]:
embassy_url_map['usau'] = 'https://www.usau.usmission.gov/'
embassy_url_map['taiwan-2'] = 'https://www.ait.org.tw'

In [65]:
asean_request = requests.get('https://www.usembassy.gov/asean/')
asean_html = asean_request.content
asean_soup = BeautifulSoup(asean_html, 'lxml')
list(map(lambda x: x['href'], asean_soup.find_all('a')))

['https://www.usembassy.gov/',
 'https://www.usembassy.gov/',
 '#main',
 'https://www.usembassy.gov/',
 'https://www.usembassy.gov/',
 'https://asean.usmission.gov/mission/',
 'https://www.usembassy.gov/author/bottsc/',
 'https://www.usembassy.gov/category/a-en/',
 'https://www.usembassy.gov/usau/',
 'https://www.usembassy.gov/usau/',
 'https://www.usembassy.gov/afghanistan/',
 'https://www.usembassy.gov/afghanistan/',
 'https://www.usembassy.gov/seychelles/',
 'https://www.usembassy.gov/seychelles/',
 'https://www.usembassy.gov/korea/',
 'https://www.usembassy.gov/korea/',
 'https://www.usembassy.gov/curacao/',
 'https://www.usembassy.gov/curacao/',
 'https://www.usembassy.gov/us-icao/',
 'https://www.usembassy.gov/us-icao/',
 'https://www.usembassy.gov/san-marino-2/',
 'https://www.usembassy.gov/san-marino-2/',
 'http://www.state.gov/',
 'https://travel.state.gov/',
 'https://www.state.gov/privacy-policy/']

In [101]:
import json
with open("embassy_url_map.json", 'w') as f:
    json.dump(embassy_url_map, f, indent=6)

In [106]:
sitemap_url = f"{embassy_url_map['south-africa']}/post-sitemap.xml"
sitemap_request = requests.get(sitemap_url)
sitemap_html = sitemap_request.content
sitemap_soup = BeautifulSoup(sitemap_html, 'lxml')
embassy_posts = [loc.string for loc in sitemap_soup.find_all('loc')]
embassy_posts

['https://za.usembassy.gov/pepfar-nike-foundation-and-gates-foundation-partnership-restores-hope-for-an-aids-free-future-for-girls/',
 'https://za.usembassy.gov/long-fight-with-tb-the-story-of-thabo-pelesane/',
 'https://za.usembassy.gov/the-power-of-partnership-extraordinary-progress-lessons-learned-and-great-hope-for-the-future-in-south-africa/',
 'https://za.usembassy.gov/pepfar-stories-of-survival-in-rural-mpumalanga-province/',
 'https://za.usembassy.gov/siyayinqoba-beat-it-conquers-fear-of-hiv-testing/',
 'https://za.usembassy.gov/desmond-tutu-blesses-panel-from-aids-quilt-returning-to-washington/',
 'https://za.usembassy.gov/thibela-tb-has-miners-covered/',
 'https://za.usembassy.gov/icap-leaves-proud-legacy-in-eastern-cape/',
 'https://za.usembassy.gov/pepfar-partner-egpafs-successful-project-transitions-to-south-african-organizations/',
 'https://za.usembassy.gov/sm_090415/',
 'https://za.usembassy.gov/sm_200815/',
 'https://za.usembassy.gov/updated-worldwide-caution/',
 'http

In [107]:
for embassy_link in embassy_posts[:5]:
    r_post = requests.get(embassy_link)
    html_post = r_post.content
    soup_post = BeautifulSoup(html_post, 'lxml')
    
    post_content = ''
    
    post_content += (soup_post.find(class_='mo-breadcrumbs').find('h1').string.strip()) + " "
    for sibling in soup_post.find(class_='main').article.find(class_='entry-content').div.next_siblings:
        if type(sibling) is bs4.element.Tag:
            if sibling.name == 'p':
                if sibling
                post_content += f"{sibling.text} "
    print(post_content+"\n")
    time.sleep(2)

PEPFAR, Nike Foundation and Gates Foundation Partnership Restores Hope for an AIDS-free Future for Girls     Ask an adolescent girl about her dreams for the future and hopefully you will hear bold ambitions: dreams of becoming a doctor, lawyer, scientist or a teacher. You might learn of dreams to travel the world or dreams to one day get married and start a family. You will never hear a girl dream of becoming infected with HIV. But this week alone, more than 7,000 adolescent girls and young women — future doctors, lawyers, teachers, travelers and mothers — will become infected with the virus. HIV/AIDS is the leading cause of death among adolescent girls and young women age 15-24 in sub-Saharan Africa. Young women and girls in the region are twice as likely to be living with HIV as boys and men the same age, and account for one in four new infections in the region. These are tragic and troubling statistics that stand in contrast to the considerable progress the global community has made

PEPFAR Stories of Survival in Rural Mpumalanga Province A person needs to spend only a moment speaking with a staff member at the Amsterdam Community Health Centre (CHC) or one of the 1,430 patients currently receiving antiretroviral therapy (ART) at the facility to understand the benefits of Nurse Initiated and Managed Antiretroviral Treatment (NIMART). Caring nurses like BroadReach’s Nurse Mentor Elza Marx tops the chart. Thanks to assistance from PEFPAR, the ability for nurses to initiate HIV-positive patients onto ART and mentor other cadres of healthcare professionals has irrevocably changed not only HIV and AIDS services offered in facilities like Amsterdam throughout South Africa, but also the hearts and minds of countless patients receiving this vital care. Situated in rural South Africa, less than 20 kilometres from the Swaziland border in eastern Mpumalanga, Amsterdam CHC serves a population of approximately 24,000 people; with 3,000 to 4,000 patients visiting the facility ea

In [117]:
print(soup_post.find(class_='mo-breadcrumbs').find('h1').string.strip())
for sibling in soup_post.find(class_='main').article.find(class_='entry-content').div.next_siblings:
    if type(sibling) is bs4.element.Tag:
        if sibling.name == 'p':
            if sibling.attrs:
                if 'class' in sibling.attrs:
                    if 'byline' not in sibling.attrs['class']:
                        print(sibling.text if sibling.text else '')
            else:
                print(sibling.text if sibling.text else '')

Siyayinqoba: Beat It! Conquers Fear of HIV Testing
Siyayinqoba Beat It!, a South African magazine show produced by the Community Health Media Trust (CMT) discusses hard-hitting topics about people living with HIV.
Not content with using a mass media to reach out on a national level, CMT goes one step further: with funding from PEPFAR through the Centers for Disease Control and Prevention the team has rolled out a comprehensive social mobilization campaign using the catchy Siyayinqoba brand at all levels of media, starting at national and reaching out to local communities through thought-provoking radio talk shows, news articles and even taxis branded with HIV prevention messaging.
CMT’s new project team, based in rural Mthatha in the Eastern Cape, works with local NGOs and community platforms to create a communication cascade that ensures the flow of HIV prevention and health information to all corners of the communities in the predominantly isiXhosa province.
CMT works closely with TB

In [172]:
for s in soup_post.find("main").article.find(class_='entry-content').div.next_siblings:
    print(s.name, s.string, type(s))
    if s.name == 'p':
        if 'class' in s.attrs:
            print('Attributes:', s.attrs['class'])
            if 'byline' not in s.attrs['class']:
                print('Hello!', s.text)
        else:
            print('Hello!', s.text)
        print()

figure None <class 'bs4.element.Tag'>
None 
 <class 'bs4.element.NavigableString'>
p None <class 'bs4.element.Tag'>
Hello! Della Mae Live in Majuro! Embassy Majuro welcomed Nashville-based bluegrass band Della Mae with traditional Marshallese songs and dances performed by Youth to Youth in Health.  Band members spent the week holding music workshops with local youth around town.  Their visit will culminate Saturday night with a free all ages concert in Delap Park at 7 pm.  Della Mae’s visit to Majuro is part of the American Music Abroad program.

None 
 <class 'bs4.element.NavigableString'>
None  AddThis Advanced Settings above via filter on the_content  <class 'bs4.element.Comment'>
None  AddThis Advanced Settings below via filter on the_content  <class 'bs4.element.Comment'>
None  AddThis Advanced Settings generic via filter on the_content  <class 'bs4.element.Comment'>
None  AddThis Share Buttons above via filter on the_content  <class 'bs4.element.Comment'>
None  AddThis Share Butt

In [171]:
'byline' in soup_post.find("main").article.find(class_='entry-content').find(class_='byline').attrs['class']

True

In [162]:
soup_post.find("main").article.find(class_='entry-content').attrs

{'class': ['entry-content']}